<a href="https://colab.research.google.com/github/aeyjeyaryan/Deep-Learning/blob/main/Sentiment_Analysis_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

In [4]:
df = pd.read_csv("sentiment_dataset.csv")

In [5]:
df.head()

,text,sentiment
0,"I love this product, it's amazing!",Positive
1,This is the worst thing I have ever bought.,Negative
2,I am extremely happy with my purchase.,Positive
3,"I hate it, it's terrible.",Negative
4,This is the best purchase I have ever made.,Positive


In [6]:
# Importing libraries
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
# dropout layer that drops entire 1D feature maps instead of individual elements.
#  SpatialDropout1D: Regularizes the model by dropping entire feature maps.

from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


In [7]:
# Basic preprocessing
df['text'] = df['text'].apply(lambda x: x.lower())

# Tokenizing the text
tokenizer = Tokenizer(num_words=5000, lower=True)
tokenizer.fit_on_texts(df['text'].values)
X = tokenizer.texts_to_sequences(df['text'].values)
X = pad_sequences(X, maxlen=100)

# Encode the target variable
Y = pd.get_dummies(df['sentiment']).values

# Train-test split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [8]:
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=100, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [9]:
epochs = 5
batch_size = 64

early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=2)

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2, callbacks=[early_stopping])


Epoch 1/5
1/1 [==============================] - 5s 5s/step - loss: 0.6939 - accuracy: 0.5000 - val_loss: 0.6897 - val_accuracy: 0.5000
Epoch 2/5
1/1 [==============================] - 0s 401ms/step - loss: 0.6864 - accuracy: 1.0000 - val_loss: 0.6913 - val_accuracy: 0.5000
Epoch 3/5
1/1 [==============================] - 0s 407ms/step - loss: 0.6857 - accuracy: 0.8333 - val_loss: 0.6929 - val_accuracy: 0.5000
Epoch 4/5
1/1 [==============================] - 0s 325ms/step - loss: 0.6689 - accuracy: 1.0000 - val_loss: 0.6945 - val_accuracy: 0.5000
Epoch 4: early stopping


In [10]:
# Evaluate on test set
loss, accuracy = model.evaluate(X_test, Y_test, verbose=2)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

# Get predictions
Y_pred = model.predict(X_test)
Y_pred_classes = np.argmax(Y_pred, axis=1)
Y_true = np.argmax(Y_test, axis=1)

print(classification_report(Y_true, Y_pred_classes, target_names=['Negative', 'Positive']))


1/1 - 0s - loss: 0.7031 - accuracy: 0.0000e+00 - 37ms/epoch - 37ms/step
Test Accuracy: 0.00%
1/1 [==============================] - 0s 460ms/step
              precision    recall  f1-score   support

    Negative       0.00      0.00      0.00       1.0
    Positive       0.00      0.00      0.00       1.0

    accuracy                           0.00       2.0
   macro avg       0.00      0.00      0.00       2.0
weighted avg       0.00      0.00      0.00       2.0



In [23]:
input_user = input("Enter your text: ")

sequence = tokenizer.texts_to_sequences([input_user])
padded_sequence = pad_sequences(sequence, maxlen=100)

prediction = model.predict(padded_sequence)


if prediction[0][0] > 0.5:
    print("Positive")
else:
    print("Negative")

Enter your text: this is the worst thing that I have ever bought
1/1 [==============================] - 0s 51ms/step
Negative
